<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Hamiltonian and momentum constraint equations using NRPyLaTeX 

## Author: Thiago Assumpção


## In this tutorial, we check that the UIUC metric satisfies the Hamiltonian and momentum constraint equations. We begin with the manual approach, and subsequently demonstrate how to reproduce the results using NRPyLaTeX.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules
1. [Step 2](#uiucbh): UIUC metric
1. [Step 3](#constraints): Constraint equations with `NRPyLaTeX`
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

In [1]:
import sympy as sp                       # SymPy: The Python computer algebra package upon which NRPy+ depends
import indexedexp as ixp                 # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import NRPy_param_funcs as par           # NRPy+: Parameter interface
from UnitTesting.assert_equal import check_zero # NRPy+: unit testing suite

In [2]:
!pip install nrpylatex~=1.2 > /dev/null
!pip freeze | grep nrpylatex

from nrpylatex import parse_latex
%load_ext nrpylatex.extension

nrpylatex==1.2.2


<a id='uiucbh'></a>

# Step 2: UIUC metric \[Back to [top](#toc)\]
$$\label{uiucbh}$$

Here, we denote the Boyer-Lindquist coordinates by ($r_{\rm BL}$, $\theta$, $\phi$). After two successive radial transformations (introduced by [Brandt & Seidel, PRD 54 1403, 1996](https://arxiv.org/abs/gr-qc/9601010)) and [Liu, Etienne, & Shapiro, PRD 80 121503, 2009](https://arxiv.org/abs/1001.4077)), the spatial sector of the Kerr metric takes the following form:

$$
^{(3)}ds^2 = \gamma_{ij}dx^i dx^j = \frac{ \Sigma (r + \frac{r_{+}}{4})^2}{ r^3 (r_{BL} - r_{-} )} dr^2
           + \Sigma d\theta^2  +  \frac{A \sin^2\theta}{\Sigma} d\phi^2 \,,
$$

where

$$
\begin{aligned}
r_{\pm}  &= M \pm \sqrt{M^2 - a^2} \,, \\
r_{BL} &= r\left(1 + \frac{ r_{+} }{4r}\right)^2 \,, \\
\Sigma &= r_{BL}^2 + a^2 \cos^2 \theta \,, \\
\Delta &= r_{BL}^2 - 2 M r_{BL} + a^2 \,, \\
A &= (r_{BL}^2 + a^2)^2 - \Delta a^2 \sin^2 \theta  \,.
\end{aligned}
$$

The conformal factor is given by (see, e.g., Eq. (2) in [Liu, Etienne, & Shapiro, PRD 80 121503, 2009](https://arxiv.org/abs/1001.4077) or Eq. (4a) in [Brandt & Seidel, PRD 54 1403, 1996](https://arxiv.org/abs/gr-qc/9601010)):

$$
\psi^4 = \frac{A}{\Sigma} \,.
$$

The lapse function and the non-vanishing component of the shift vector are given by:
$$
\begin{aligned}
\alpha &= \sqrt{ \frac{\Delta \Sigma} {A} } \,, \\
\beta^{\phi} &= - \frac{ 2aM r_{BL} } {A} \,. \\
\end{aligned}
$$



<a id='constraints'></a>

# Step 3: Constraint equations with `NRPyLaTeX` \[Back to [top](#toc)\]
$$\label{constraints}$$

In this section, we use `NRPyLaTeX` to compute the conformal constraint equations with the above metric. We begin by inserting the metric element, the ADM variables, the conformal factor, and the conformal metric. We then construct the extrinsic curvatures and conformal Ricci scalar, and compute the constraint equations:

In [3]:
%%parse_latex --reset %%--debug

%% define LES coordinates
% coord [r, \theta, \phi]

%% define any relevant constant(s)
% define a M --const

%% initialize every component of the metric to zero
% define gammaDD --dim 3 --zero

%% initialize every component of the conformal metric to zero
% define gammatildeDD --dim 3 --zero

%% initialize extrinsic curvature tensor KDD
% define KDD --dim 3 --zero

%% initialize shift vector betaU
% define betaU --dim 3 --zero

%% initialize momentum constraint violation vector MU
% define mU --dim 3 --zero

%% define string replacements
% srepl "rp"  -> "\text{rp}"
% srepl "rm"  -> "\text{rm}"
% srepl "r_{-}"  -> "\text{rp}"
% srepl "r_{+}"  -> "\text{rm}"
% srepl "rBL" -> "\text{rBL}"
% srepl "r_{BL}" -> "\text{rBL}"

% ignore "\begin{align}" "\end{align}"
\begin{align}
%% Define quantities that are used to construct the metric elements
r_{+}  &= M + \sqrt{M^2 - a^2} \\ %% Boyer-Lindquist outer horizon
r_{-}  &= M - \sqrt{M^2 - a^2} \\ %% Boyer-Lindquist inner horizon
r_{BL} &= r\left(1 + \frac{ r_{+} }{4r}\right)^2 \\
\Sigma &= r_{BL}^2 + a^2 \cos^2 \theta \\
\Delta &= r_{BL}^2 - 2 M r_{BL} + a^2 \\
A &= (r_{BL}^2 + a^2)^2 - \Delta a^2 \sin^2 \theta \\
%% Define metric elements
\gamma_{r r} &= \frac{ \Sigma (r + \frac{r_{+}}{4})^2}{ r^3 (r_{BL} - r_{-} )} \\ %% \gamma_{r r} = g_{0 0}
\gamma_{\theta\theta} &= \Sigma \\
\gamma_{\phi\phi} &= \frac{A \sin^2\theta}{\Sigma} \\
%% generate inverse gammaUU, determinant det(gammaDD), and connection GammaUDD
% assign gammaDD --metric
%% Define ADM variables
\alpha &= \sqrt{ \frac{\Delta \Sigma} {A} } \\
\beta^{\phi} &= - \frac{ 2aM r_{BL} } {A} \\
%% Define conformal factor
\psi &= \left(\frac{A}{\Sigma}\right)^{1/4} \\
%% Define conformal metric
\tilde{\gamma}_{ij} &= \psi^{{-4}} \gamma_{ij} \\
%% generate inverse gammatildeUU, determinant det(gammatildeDD), and connection GammatildeUDD
% assign gammatildeDD --metric
%% Define extrinsic curvature
K_{ij} &= \frac{1}{2\alpha} \left( \nabla_{i}\beta_{j} + \nabla_{j}\beta_{i} \right) \\
%% Trace of extrinsic curvature
K &= \gamma^{ij} K_{ij}  \\
%% Tracefree extrinsic curvature
A_{ij} &= K_{ij} - \frac{1}{3} K \gamma_{ij} \\
%% Conformal, tracefree extrinsic curvature
\tilde{A}_{ij} &= \psi^2 A_{ij} \\
%% Ricci tensor
\tilde{R}_{ij} &= -\frac{1}{2} \tilde{\gamma}^{kl} ( \tilde{\gamma}_{ij,lk} 
                  + \tilde{\gamma}_{kl,ji} - \tilde{\gamma}_{kj,li} - \tilde{\gamma}_{il,jk})
                  + \tilde{\gamma}^{kl} ( \tilde{\Gamma}^{m}_{il} \tilde{\Gamma}_{mkj} 
                  - \tilde{\Gamma}^{m}_{ij} \tilde{\Gamma}_{mkl}  ) \\
%% Ricci scalar
\tilde{R} &= \tilde{\gamma}^{ij} \tilde{R}_{ij} \\
%% Hamiltonian contraint violation
% srepl "\tilde{\nabla}^2" -> " \tilde{\gamma}^{ij} \tilde{\nabla}_i \tilde{\nabla}_j"
H &= \tilde{\nabla}^2 \psi - \frac{\psi \tilde{R}}{8} - \frac{ \psi^5 K^{{2}} }{12} 
   + \frac{\tilde{A}_{ij}\tilde{A}^{ij}}{8\psi^7} \\
%% Momentum constraint violation
%% srepl "\tilde{\nabla}^i" -> "\tilde{\gamma}^{ij} \tilde{\nabla}_j"
m^{i} &= \tilde{\nabla}_j \tilde{A}^{ij} - \frac{2}{3} \psi^6 \tilde{\gamma}^{ij} \tilde{\nabla}_j K
\end{align}

('rm',
 'RtildeDD',
 'Rtilde',
 'AtildeUU',
 'gammadet',
 'gammatildeDD',
 'psi_dDD',
 'GammatildeDDD',
 'psi_dD',
 'GammaUDD',
 'betaU',
 'rp',
 'ADD',
 'AtildeDD',
 'gammaUU',
 'KDD',
 'K_cdtildeD',
 'rBL',
 'GammatildeUDD',
 'gammaDD',
 'M',
 'gammatildedet',
 'Delta',
 'betaD',
 'H',
 'epsilonUUU',
 'K',
 'psi_cdtildeD_dD',
 'psi_cdtildeDD',
 'betaD_cdD',
 'Sigma',
 'mU',
 'alpha',
 'a',
 'psi',
 'gammatildeUU',
 'r',
 'A',
 'psi_cdtildeD',
 'theta',
 'AtildeUU_cdtildeD')

Once the quantities have been constructed, we can use the `check_zero` from the `UnitTesting.assert_equal` module to check if they are identically zero:

In [4]:
# Check momemtum constraints
for i in range(3):
    print( check_zero(mU[i]) )

True
True
True


In [5]:
# Check Hamiltonian constraint
print( check_zero(H) )

True


Alternatively, we can use the `parse_latex` function. This is especially useful if the we want to run it in a `.py` file.

In [6]:
parse_latex(r"""
%% define LES coordinates
% coord [r, \theta, \phi]

%% define any relevant constant(s)
% define a M --const

%% initialize every component of the metric to zero
% define gammaDD --dim 3 --zero

%% initialize every component of the conformal metric to zero
% define gammatildeDD --dim 3 --zero

%% initialize extrinsic curvature tensor KDD
% define KDD --dim 3 --zero

%% initialize shift vector betaU
% define betaU --dim 3 --zero

%% initialize momentum constraint violation vector MU
% define mU --dim 3 --zero

%% define string replacements
% srepl "rp"  -> "\text{rp}"
% srepl "rm"  -> "\text{rm}"
% srepl "r_{-}"  -> "\text{rp}"
% srepl "r_{+}"  -> "\text{rm}"
% srepl "rBL" -> "\text{rBL}"
% srepl "r_{BL}" -> "\text{rBL}"

% ignore "\begin{align}" "\end{align}"
\begin{align}
%% Define quantities that are used to construct the metric elements
r_{+}  &= M + \sqrt{M^2 - a^2} \\ %% Boyer-Lindquist outer horizon
r_{-}  &= M - \sqrt{M^2 - a^2} \\ %% Boyer-Lindquist inner horizon
r_{BL} &= r\left(1 + \frac{ r_{+} }{4r}\right)^2 \\
\Sigma &= r_{BL}^2 + a^2 \cos^2 \theta \\
\Delta &= r_{BL}^2 - 2 M r_{BL} + a^2 \\
A &= (r_{BL}^2 + a^2)^2 - \Delta a^2 \sin^2 \theta \\
%% Define metric elements
\gamma_{r r} &= \frac{ \Sigma (r + \frac{r_{+}}{4})^2}{ r^3 (r_{BL} - r_{-} )} \\ %% \gamma_{r r} = g_{0 0}
\gamma_{\theta\theta} &= \Sigma \\
\gamma_{\phi\phi} &= \frac{A \sin^2\theta}{\Sigma} \\
%% generate inverse gammaUU, determinant det(gammaDD), and connection GammaUDD
% assign gammaDD --metric
%% Define ADM variables
\alpha &= \sqrt{ \frac{\Delta \Sigma} {A} } \\
\beta^{\phi} &= - \frac{ 2aM r_{BL} } {A} \\
%% Define conformal factor
\psi &= \left(\frac{A}{\Sigma}\right)^{1/4} \\
%% Define conformal metric
\tilde{\gamma}_{ij} &= \psi^{{-4}} \gamma_{ij} \\
%% generate inverse gammatildeUU, determinant det(gammatildeDD), and connection GammatildeUDD
% assign gammatildeDD --metric
%% Define extrinsic curvature
K_{ij} &= \frac{1}{2\alpha} \left( \nabla_{i}\beta_{j} + \nabla_{j}\beta_{i} \right) \\
%% Trace of extrinsic curvature
K &= \gamma^{ij} K_{ij}  \\
%% Tracefree extrinsic curvature
A_{ij} &= K_{ij} - \frac{1}{3} K \gamma_{ij} \\
%% Conformal, tracefree extrinsic curvature
\tilde{A}_{ij} &= \psi^2 A_{ij} \\
%% Ricci tensor
\tilde{R}_{ij} &= -\frac{1}{2} \tilde{\gamma}^{kl} ( \tilde{\gamma}_{ij,lk} 
                  + \tilde{\gamma}_{kl,ji} - \tilde{\gamma}_{kj,li} - \tilde{\gamma}_{il,jk})
                  + \tilde{\gamma}^{kl} ( \tilde{\Gamma}^{m}_{il} \tilde{\Gamma}_{mkj} 
                  - \tilde{\Gamma}^{m}_{ij} \tilde{\Gamma}_{mkl}  ) \\
%% Ricci scalar
\tilde{R} &= \tilde{\gamma}^{ij} \tilde{R}_{ij} \\
%% Hamiltonian contraint violation
% srepl "\tilde{\nabla}^2" -> " \tilde{\gamma}^{ij} \tilde{\nabla}_i \tilde{\nabla}_j"
H &= \tilde{\nabla}^2 \psi - \frac{\psi \tilde{R}}{8} - \frac{ \psi^5 K^{{2}} }{12} 
   + \frac{\tilde{A}_{ij}\tilde{A}^{ij}}{8\psi^7} \\
%% Momentum constraint violation
%% srepl "\tilde{\nabla}^i" -> "\tilde{\gamma}^{ij} \tilde{\nabla}_j"
m^{i} &= \tilde{\nabla}_j \tilde{A}^{ij} - \frac{2}{3} \psi^6 \tilde{\gamma}^{ij} \tilde{\nabla}_j K
\end{align}
""", reset=True)

('rm',
 'RtildeDD',
 'Rtilde',
 'AtildeUU',
 'gammadet',
 'gammatildeDD',
 'psi_dDD',
 'GammatildeDDD',
 'psi_dD',
 'GammaUDD',
 'betaU',
 'rp',
 'ADD',
 'AtildeDD',
 'gammaUU',
 'KDD',
 'K_cdtildeD',
 'rBL',
 'GammatildeUDD',
 'gammaDD',
 'M',
 'gammatildedet',
 'Delta',
 'betaD',
 'H',
 'epsilonUUU',
 'K',
 'psi_cdtildeD_dD',
 'psi_cdtildeDD',
 'betaD_cdD',
 'Sigma',
 'mU',
 'alpha',
 'a',
 'psi',
 'gammatildeUU',
 'r',
 'A',
 'psi_cdtildeD',
 'theta',
 'AtildeUU_cdtildeD')

Notice that the `parse_latex` function provides us with a list of all the variables that have been internally parsed by `NRPyLaTeX`.

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Hamiltonian-and-momentum-constraint-equations-using-NRPyLaTeX.pdf](Tutorial-Hamiltonian-and-momentum-constraint-equations-using-NRPyLaTeX.pdf). (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [8]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Hamiltonian-and-momentum-constraint-equations-using-NRPyLaTeX")

Created Tutorial-Hamiltonian-and-momentum-constraint-equations-using-
    NRPyLaTeX.tex, and compiled LaTeX file to PDF file Tutorial-
    Hamiltonian-and-momentum-constraint-equations-using-NRPyLaTeX.pdf
